In [98]:
# Import library modules

import os
import cv2
import imutils
import glob
import numpy as np
import math
import matplotlib.pyplot as plt
import scipy
import scipy.ndimage as ndi
from sklearn.metrics import mean_squared_error
from scipy.signal import find_peaks
from Img.filters import *
from Puzzle.Puzzle import *
from Puzzle.Enums import *

In [99]:
import warnings
warnings.filterwarnings("ignore")

In [100]:
# Read input image files

filename = "../images/input/*.jpg"

img_color = [cv2.imread(file, cv2.IMREAD_COLOR) for file in glob.glob(filename)][0]

In [101]:
# Create binary mask for each image

# The kernel to be used for dilation purpose
kernel = np.ones((3, 3), np.uint8)
  
# defining the lower and upper values of HSV to detect green colour
Lower_hsv = np.array([45, 10, 10])
Upper_hsv = np.array([95, 255, 255])

# converting the image to HSV format
hsv = cv2.cvtColor(img_color, cv2.COLOR_BGR2HSV)

# creating the mask by eroding, morphing, dilating process
Mask = cv2.inRange(hsv, Lower_hsv, Upper_hsv)
Mask = cv2.medianBlur(Mask, 21)
Mask = cv2.erode(Mask, kernel, iterations=1)
Mask = cv2.morphologyEx(Mask, cv2.MORPH_OPEN, kernel, iterations=1)
Mask = cv2.dilate(Mask, kernel, iterations=1)

# Inverting the mask by performing bitwise-not operation
Mask = cv2.bitwise_not(Mask)
img_mask = cv2.bitwise_and(img_color, img_color, mask=Mask)

cv2.imwrite('../images/output/img_mask.jpg', img_mask)

# Find contours

contours, hierarchy = cv2.findContours(Mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

contours_largest_perimeter = []

contours = sorted(contours, key=cv2.contourArea, reverse=True)
max_contour = contours[0].shape[0]
contours_largest_perimeter = [contour for contour in contours if contour.shape[0] > max_contour / 3]

puzzle_piece_cnt = len(contours_largest_perimeter)
print(f'No. of largest perimeter contours detected: {puzzle_piece_cnt}')

No. of largest perimeter contours detected: 20


In [102]:
# Solve puzzle

Puzzle(img_mask, Mask, contours_largest_perimeter)

>>> START solving puzzle
Possible sizes: ( 20 pieces with 14 borders among them): [(4, 5), (5, 4)]
Number of border pieces:  14
>>> START solve border
<--- New match ---> pieces left:  13 extremum: (0, 0, 1, 1) puzzle dimension: [(4, 5), (5, 4)]
Placed: TypePiece.BORDER at (1, 0)
<--- New match ---> pieces left:  12 extremum: (0, 0, 2, 1) puzzle dimension: [(4, 5), (5, 4)]
Placed: TypePiece.BORDER at (2, 0)
<--- New match ---> pieces left:  11 extremum: (0, 0, 3, 1) puzzle dimension: [(4, 5), (5, 4)]
Update possible dimensions with extremum (0, 0, 4, 1) : [(5, 4)]
Placed: TypePiece.BORDER at (3, 0)
<--- New match ---> pieces left:  10 extremum: (0, 0, 4, 1) puzzle dimension: [(5, 4)]
Placed: TypePiece.BORDER at (0, 1)
<--- New match ---> pieces left:  9 extremum: (0, 0, 4, 2) puzzle dimension: [(5, 4)]
Placed: TypePiece.BORDER at (0, 2)
<--- New match ---> pieces left:  8 extremum: (0, 0, 4, 3) puzzle dimension: [(5, 4)]
Placed: TypePiece.ANGLE at (0, 3)
<--- New match ---> pieces left